# main.ipynb

## modules

### common

In [ ]:
import os
import shutil
import json
from uuid import uuid4
from pathlib import Path

### requirements

In [ ]:
import requests
import pendulum
import pandas as pd
import fire

## 1. pendulum

In [ ]:
# > variables
region_city: str = 'America/Lima'

In [ ]:
c_date = pendulum.now(rc := region_city).to_date_string()

print(c_date, '---', rc)

## 2. requests

In [ ]:
# > variables
serie = 'PM04902AA'
api_url = f'https://estadisticas.bcrp.gob.pe/estadisticas/series/api/{serie}/json/2020/2022/'
headers = {'Content-Type': 'application/json'}

In [ ]:
# > request + try-except
try:
    response = requests.get(url=api_url, headers=headers, timeout=60)
    response.encoding = 'utf-8'
    api_data = json.loads(response.text)
except Exception as e:
    print('[INFO] something went wrong...')
    raise

In [ ]:
print(json.dumps(api_data, indent=2))

## 3. pandas: dataframes

In [ ]:
# > variables
record_path = 'periods'
columnas = {'name': 'YEAR', 'values': serie.upper()}

In [ ]:
# > pandas from json
df = pd.json_normalize(api_data, record_path=record_path)

df.head()

In [ ]:
# > pandas rename column
df = df.rename(columns=columnas)

df.head()

In [ ]:
# > pandas format
df[serie] = df[serie].str[0].astype('float')

df.head()

## 4. pandas: exporting data

In [ ]:
# > variables
curr_date = c_date
exec_uuid = str(uuid4())
s = serie

save_path = f'./data/current/{curr_date}/{exec_uuid}/{s}.csv'

print(save_path)

In [ ]:
# > split save path
save_folder = '/'.join(save_path.split('/')[:-1])

print(save_folder)

In [ ]:
# > make save directory if not exists
Path(save_folder).mkdir(parents=True, exist_ok=True)

In [ ]:
# > export data from dataframe to csv
dfc = df.copy(deep=True)
dfc.to_csv(save_path, sep=';', encoding='iso-8859-1', index=False)

## 5. main scenario

### 5.1. multiple solicitudes

In [ ]:
# > fixed
series = ['PM04901AA', 'PM04902AA', 'PM04903AA', 'PM04904AA', 'PM04905AA', 'PM04906AA', 'PM04907AA']
exec_uuid = str(uuid4())

for serie in series:
    # > variables
    api_url = f'https://estadisticas.bcrp.gob.pe/estadisticas/series/api/{serie}/json/2020/2022/'
    headers = {'Content-Type': 'application/json'}
    
    # > request + try-except
    try:
        response = requests.get(url=api_url, headers=headers, timeout=60)
        response.encoding = 'utf-8'
        api_data = json.loads(response.text)
    except Exception as e:
        print('[INFO] something went wrong...')
        raise
    
    # > variables
    record_path = 'periods'
    columnas = {'name': 'YEAR', 'values': serie.upper()}
    
    # > pandas from json
    df = pd.json_normalize(api_data, record_path=record_path)
    df = df.rename(columns=columnas)
    df[serie] = df[serie].str[0].astype('float')
    
    # > variables
    curr_date = c_date
    s = serie
    save_path = f'./data/current/{curr_date}/{exec_uuid}/{s}.csv'
    
    # > split save path
    save_folder = '/'.join(save_path.split('/')[:-1])
    
    # > make save directory if not exists
    Path(save_folder).mkdir(parents=True, exist_ok=True)
    
    # > export data from dataframe to csv
    dfc = df.copy(deep=True)
    dfc.to_csv(save_path, sep=';', encoding='iso-8859-1', index=False)

### 5.2. dataframe auxiliar

In [ ]:
# > variables
curr_date = c_date ; year = curr_date.split('-')[0]
read_path = '/'.join(save_path.split('/')[:-1])

In [ ]:
# > dataframe dummy
tdf = pd.DataFrame({'YEAR': range(1940, int(year)+1), 'LOAD_DATE': curr_date})
tdf = tdf.set_index('YEAR')

tdf.head()

In [ ]:
# > elementos a juntar
files = os.listdir(read_path)
for f in files:
    print(f)

In [ ]:
# > complete table
files = os.listdir(read_path)
for f in files:
    df = pd.read_csv(f'{read_path}/{f}', sep=';', encoding='iso-8859-1')
    df = df.set_index('YEAR')
    tdf = tdf.join(df)

In [ ]:
# > nulos por dummy table
print(tdf.isna().sum())
tdf.head()

In [ ]:
# > reset index + drop nulos
tdf = tdf.reset_index()
tdf = tdf.dropna(subset=tdf.columns[2:])

In [ ]:
# > output
print(tdf.isna().sum())
tdf.head()

### 5.3. exportar datos

In [ ]:
# > variables
export_path = f'./data/output/{curr_date}/output_{exec_uuid}.csv'
save_folder = '/'.join(export_path.split('/')[:-1])
sep = '|'
encoding='iso-8859-1'

In [ ]:
# > export full table
Path(save_folder).mkdir(parents=True, exist_ok=True)
df.to_csv(export_path, sep=sep, encoding=encoding, index=False)

In [ ]:
# > move current to historic
current_path = read_path
historic_path = read_path.replace('current', 'historic')
shutil.move(current_path, historic_path)